# Example of MCAO interaction matrices

## Off target interaction matrix - One layer, two NGSs
We compute the interaction matrix for a system measuring tip, tilt and focus with two NGS and correcting tip, tilt and focus. As a first step, we consider only one layer - consequently, there will be only one DM. <br> We compare the elements of the interaction matrix with the analytical ones from Negro84.

In [2]:
from apposto.utils.footprint_geometry import FootprintGeometry
from apposto.types.aperture import CircularOpticalAperture
from apposto.utils.mcao_interaction_matrix import MCAOInteractionMatrix
import numpy as np

In [23]:
ngs1_pos = (30, 0)
ngs2_pos = (-10, 20)
target_pos = (0, 0)
zenith_angle = 0
telescope_radius = 10
fov = 50
altitude = 10e3
n_modes_sensed = 3
idx_modes_corrected = [np.array([2, 3, 4])]

fp = FootprintGeometry()
fp.set_zenith_angle(zenith_angle)
fp.setTelescopeRadiusInMeter(telescope_radius)
fp.setInstrumentFoV(fov)
fp.setLayerAltitude(altitude)
fp.addNgs(ngs1_pos[0], ngs1_pos[1])
fp.addNgs(ngs2_pos[0], ngs2_pos[1])
fp.addTarget(target_pos[0], target_pos[1])
fp.compute()

In [24]:
metapupil_radius = fp.getMetapupilFootprint()[0].r
ngs1_footprint_radius = fp.getNgsFootprint()[0].r
ngs2_footprint_radius = fp.getNgsFootprint()[1].r
print('Metapupil radius in meters: {}'.format(metapupil_radius))
print('NGS1 radius in meters: {}'.format(ngs1_footprint_radius))
print('NGS2 radius in meters: {}'.format(ngs2_footprint_radius))

Metapupil radius in meters: 11.4544
NGS1 radius in meters: 10
NGS2 radius in meters: 10


In [25]:
dm = CircularOpticalAperture(metapupil_radius, [0, 0, 0])

In [33]:
im = MCAOInteractionMatrix([fp], [dm], n_modes_sensed, idx_modes_corrected)
ngs_im = im.getInteractionMatrixOffTarget()
print('Interaction matrix: \n {}'.format(ngs_im))

Interaction matrix: 
 [[ 8.73362445e-01  0.00000000e+00  3.83131316e-01]
 [ 1.10480201e-16  8.73362445e-01  4.44089210e-16]
 [-1.96457434e-16  0.00000000e+00  7.62761939e-01]
 [ 8.73362445e-01 -1.08420217e-19 -1.18902822e-01]
 [ 1.16876994e-16  8.73362445e-01 -3.96342741e-02]
 [-1.87892236e-16  0.00000000e+00  7.62761939e-01]]


In [45]:
fp_ngs1 = fp.getNgsFootprint()[0]
fp_ngs2 = fp.getNgsFootprint()[1]
h1 = np.sqrt(fp_ngs1.x**2 + fp_ngs1.y**2)
a1 = np.arctan2(fp_ngs1.y, fp_ngs1.x)
h2 = np.sqrt(fp_ngs2.x**2 + fp_ngs2.y**2)
a2 = np.arctan2(fp_ngs2.y, fp_ngs2.x)
r1 = ngs1_footprint_radius
r2 = ngs2_footprint_radius
R = metapupil_radius

negro22 = r1 / R
negro33 = negro22
negro44 = negro22**2
negro24_ngs1 = 2 * np.sqrt(3) * h1 / R * r1 / R * np.cos(a1) 
negro24_ngs2 = 2 * np.sqrt(3) * h2 / R * r2 / R * np.cos(a2)
negro34_ngs1 = 2 * np.sqrt(3) * h1 / R * r1 / R * np.sin(a1)
negro34_ngs2 = 2 * np.sqrt(3) * h2 / R * r2 / R * np.sin(a2)

print('22 from Negro: {}, \n22 from interaction matrix: {} \n'.format(negro22, ngs_im[0][0]))
print('33 from Negro: {}, \n33 from interaction matrix: {} \n'.format(negro33, ngs_im[1][1]))
print('44 from Negro: {}, \n44 from interaction matrix: {} \n'.format(negro44, ngs_im[2][2]))
print('24_ngs1 from Negro: {}, \n24_ngs1 from interaction matrix: {} \n'.format(negro24_ngs1, ngs_im[0][2]))
print('24_ngs2 from Negro: {}, \n24_ngs2 from interaction matrix: {} \n'.format(negro24_ngs2, ngs_im[3][2]))
print('34_ngs1 from Negro: {}, \n34_ngs1 from interaction matrix: {} \n'.format(negro34_ngs1, ngs_im[1][2]))
print('34_ngs2 from Negro: {}, \n34_ngs2 from interaction matrix: {} \n'.format(negro34_ngs2, ngs_im[4][2]))

22 from Negro: 0.8730269590724962, 
22 from interaction matrix: 0.8733624454148553 

33 from Negro: 0.8730269590724962, 
33 from interaction matrix: 0.8733624454148392 

44 from Negro: 0.7621760712673699, 
44 from interaction matrix: 0.7627619393842133 

24_ngs1 from Negro: 0.3839987394851919, 
24_ngs1 from interaction matrix: 0.3831313164851782 

24_ngs2 from Negro: -0.12028026062844177, 
24_ngs2 from interaction matrix: -0.11890282235746849 

34_ngs1 from Negro: 0.0, 
34_ngs1 from interaction matrix: 4.440892098500626e-16 

34_ngs2 from Negro: -0.04377843463853385, 
34_ngs2 from interaction matrix: -0.03963427411915589 



## Off target interaction matrix - Two layers, two NGS
We compute the interaction matrix for a system measuring tip and tilt with two NGS and correcting tip, tilt and focus with two DMs conjugated at layers 0km and 10km high.<br> If $n$ is the number of measured modes, if $m_1$ is the number of modes corrected on the first layer and $m_2$ the number of modes corrected on the second layer, if $N_s$ is the number of NGSs, then the dimension of the matrix must be $(n\cdot N_s)\times (m_1 + m_2)$. In this example, it must be $4x3$.

In [25]:
ngs1_pos = (30, 0)
ngs2_pos = (-10, 20)
target_pos = (0, 0)
zenith_angle = 0
telescope_radius = 10
fov = 50
altitude1 = 0
altitude2 = 10e3
n_modes_sensed = 2
idx_modes_corrected = [np.array([2, 3]), np.array([4])]

fp1 = FootprintGeometry()
fp1.set_zenith_angle(zenith_angle)
fp1.setTelescopeRadiusInMeter(telescope_radius)
fp1.setInstrumentFoV(fov)
fp1.setLayerAltitude(altitude1)
fp1.addNgs(ngs1_pos[0], ngs1_pos[1])
fp1.addNgs(ngs2_pos[0], ngs2_pos[1])
fp1.addTarget(target_pos[0], target_pos[1])
fp1.compute()

fp2 = FootprintGeometry()
fp2.set_zenith_angle(zenith_angle)
fp2.setTelescopeRadiusInMeter(telescope_radius)
fp2.setInstrumentFoV(fov)
fp2.setLayerAltitude(altitude2)
fp2.addNgs(ngs1_pos[0], ngs1_pos[1])
fp2.addNgs(ngs2_pos[0], ngs2_pos[1])
fp2.addTarget(target_pos[0], target_pos[1])
fp2.compute()

In [26]:
mp1_radius = fp1.getMetapupilFootprint()[0].r
mp2_radius = fp2.getMetapupilFootprint()[0].r
ngs1_footprint_radius = fp1.getNgsFootprint()[0].r
ngs2_footprint_radius = fp1.getNgsFootprint()[1].r
print('Metapupil radius at {}km: {}m'.format(altitude1, mp1_radius))
print('Metapupil radius at {}km: {}m'.format(altitude2/1000, mp2_radius))

Metapupil radius at 0km: 10.0m
Metapupil radius at 10.0km: 11.4544m


In [27]:
dm1 = CircularOpticalAperture(mp1_radius, [0, 0, 0])
dm2 = CircularOpticalAperture(mp2_radius, [0, 0, 0])

In [28]:
im = MCAOInteractionMatrix([fp1, fp2], [dm1, dm2], n_modes_sensed, idx_modes_corrected)
ngs_im = im.getInteractionMatrixOffTarget()
print('Interaction matrix: \n {}'.format(ngs_im))

Interaction matrix: 
 [[ 1.00000000e+00  6.17995238e-18  3.83131316e-01]
 [ 1.29453739e-16  1.00000000e+00  3.60822483e-16]
 [ 1.00000000e+00  6.17995238e-18 -1.18902822e-01]
 [ 1.29453739e-16  1.00000000e+00 -3.96342741e-02]]


## On target interaction matrix

In [38]:
ngs1 = (30, 0)
ngs2 = (-30, 0)
target = (20, 5)
zenithAngle = 0
fov = 50
telescopeRadius = 10
altitude1 = 0
altitude2 = 10e3
n_modes = 2
m_index = [np.array([2, 3]), np.array([4])]

In [39]:
fp1 = FootprintGeometry()
fp1.addNgs(ngs1[0], ngs1[1])
fp1.addNgs(ngs2[0], ngs2[1])
fp1.addTarget(target[0], target[1])
fp1.set_zenith_angle(zenithAngle)
fp1.setInstrumentFoV(fov)
fp1.setLayerAltitude(altitude1)
fp1.setTelescopeRadiusInMeter(telescopeRadius)
fp1.compute()

fp2 = FootprintGeometry()
fp2.addNgs(ngs1[0], ngs1[1])
fp2.addNgs(ngs2[0], ngs2[1])
fp2.addTarget(target[0], target[1])
fp2.set_zenith_angle(zenithAngle)
fp2.setInstrumentFoV(fov)
fp2.setLayerAltitude(altitude2)
fp2.setTelescopeRadiusInMeter(telescopeRadius)
fp2.compute()

In [40]:
dm1 = CircularOpticalAperture(telescopeRadius, [0, 0, 0])
dm2 = CircularOpticalAperture(fp2.getMetapupilFootprint()[0].r, [0, 0, 0])

In [41]:
im = MCAOInteractionMatrix([fp1, fp2], [dm1, dm2], n_modes, m_index)
im_on = im.getInteractionMatrixOnTarget()
print(im_on)

[[1.00000000e+00 6.17995238e-18 2.51017069e-01]
 [1.29453739e-16 1.00000000e+00 2.64228494e-02]]
